In [1]:
from evaluate import load
from evaluate.module import EvaluationModule
from datasets import load_dataset, Dataset
from models.qg import QG
import torch

qg = QG("the-coorporation/t5-small-qg-2.0", "the-coorporation/t5-small-qg-2.0")

## BLEU

In [2]:
bleu: EvaluationModule = load("bleu")

## METEOR

In [3]:
meteor: EvaluationModule = load("meteor")

[nltk_data] Downloading package wordnet to /home/laugu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/laugu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/laugu/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [4]:
squad: Dataset = load_dataset("the-coorporation/the_squad_qg")
validation = squad['validation']

No config specified, defaulting to: the_squad_qg/v2
Found cached dataset the_squad_qg (/home/laugu/.cache/huggingface/datasets/the-coorporation___the_squad_qg/v2/2.0.0/cd5e3386e9e3124c24114539fb5124093fa58d99c1a7292451b8801a94a2aca5)


  0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
references: list[dict] = [entry['questions'].replace(" {sep_token}", "") for entry in validation]
predictions: list[dict] = []

device = "cuda" if torch.cuda.is_available() else "cpu"

generator_args = {
    "max_length": 512,
    "num_beams": 4,
    "length_penalty": 1.5,
    "no_repeat_ngram_size": 3,
    "early_stopping": True,
}

input_strings: list[str] = ["generate questions: " + entry['context'] + " </s>" for entry in validation]
input_encodings = qg._tokenizer(input_strings, padding='longest', truncation=True, return_tensors='pt').to(device)
results = qg._model.generate(**input_encodings, **generator_args).to(device)
questions = qg._tokenizer.decode(results, skip_special_tokens=True)

OutOfMemoryError: CUDA out of memory. Tried to allocate 1.18 GiB (GPU 0; 8.00 GiB total capacity; 6.12 GiB already allocated; 241.00 MiB free; 6.15 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
# print(len(references[:5]))
# print(len(predictions))

print(references[0])
print(predictions[0])

In [ ]:
bleu.compute(references=references[:5], predictions=predictions)

In [ ]:
meteor.compute(references=references[:5], predictions=predictions)